# Problèmes

### 1)
Primal: $\underset{x \in \mathbb{R}^3}{\min} \frac{1}{2} x' Q x + c' x ~~~~~~~~ s.t. ~ A x = b ~, ~~ x \ge 0$

with  $Q = \begin{bmatrix} 6 & 2 & 1\\ 2 & 5 & 2\\ 1 & 2 & 4 \end{bmatrix} $, $c = \begin{bmatrix} -8 \\ -3 \\ -3 \end{bmatrix}$, $A = \begin{bmatrix} 1 & 0 &1 \\ 0 & 1 & 1 \end{bmatrix}$ , and $b = \begin{bmatrix} 3 \\ 0 \end{bmatrix} $.




In [1]:
using LinearAlgebra
using LaTeXStrings
using DataStructures
using BenchmarkTools

In [2]:
function display_results(result)
    # fonction pour l'affichage
    println("\n-----------------------------------------------------------------------")
    println("------------------------------- RESULTS -------------------------------")
    result
end

display_results (generic function with 1 method)

In [3]:
function Compute_AlphaAff(alpha_step, v_k, dir_v_k)
    alpha = 0
    n = length(v_k)
    while alpha+alpha_step <= 1 && all(v_k + (alpha+alpha_step) * dir_v_k .>= 0) 
        alpha += alpha_step
    end
    return alpha
end

function Compute_AlphaMax(alpha_step, v_k, dir_v_k)
    alpha = 0
    n = length(v_k)
    while alpha+alpha_step <= 100/99 && all(v_k + (alpha+alpha_step) * dir_v_k .>= 0) 
        alpha += alpha_step
    end
    return alpha
end


Compute_AlphaMax (generic function with 1 method)

In [47]:

function MehrotraPCQuad(Q, c, A, b, init_vect, max_iter, eps = 1e-10, alpha_step=1e-2, display=true)
    
    
    x_0, lambda_0, s_0 = init_vect
    x_k, lambda_k, s_k = copy(x_0), copy(lambda_0), copy(s_0)
    n_rows, n_cols = size(A) 
    
    rb_0 = A * x_0 - b
    rc_0 = -Q * x_0 + A' * lambda_0 + s_0 - c
    mu_0 = s_0' * x_0 / n_cols
    Xk = Diagonal(x_k)
    Sk = Diagonal(s_k)
    mu_k = copy(mu_0)
    rb_k = copy(rb_0)
    rc_k = copy(rc_0)
    
    k = 0
    pdd, cond_rb, cond_rc = 1, 1, 1
    e = ones(n_cols, 1)
    n_c = norm(c)
    n_b = norm(b)
    
    while k <= max_iter && pdd > eps && cond_rb > eps && cond_rc > eps  # 3 epsilon différents en pratique???

        Jacob_Fk = [-Q A' I(n_cols)
                    A zeros(n_rows, n_rows) zeros(n_rows, n_cols)
                    Sk zeros(n_cols, n_rows) Xk]
        Fk_aff = [-rc_k
                  -rb_k
                  -Xk*Sk*e]
        

        dir_aff_k = Jacob_Fk\Fk_aff
        

        alpha_aff_pri = Compute_AlphaAff(alpha_step, x_k, dir_aff_k[1:n_cols])
        alpha_aff_dual = Compute_AlphaAff(alpha_step, s_k, dir_aff_k[n_rows+n_cols+1: end])
        mu_aff = (x_k + alpha_aff_pri * dir_aff_k[1:n_cols])' * 
                    (s_k + alpha_aff_dual * dir_aff_k[n_rows+n_cols+1: end]) / n_cols
        sigma = (mu_aff / mu_k)^3
     
        Fk_cc = [zeros(n_rows+n_cols, 1)
                 sigma*mu_k*e - Diagonal(dir_aff_k[1:n_cols])*Diagonal(dir_aff_k[n_rows+n_cols+1: end])*e] 
        dir_cc_k = Jacob_Fk\Fk_cc
         
        dir_k = dir_aff_k .+ dir_cc_k
         
        alpha_max_pri = Compute_AlphaMax(alpha_step, x_k, dir_k[1:n_cols])
        alpha_max_dual = Compute_AlphaMax(alpha_step, s_k, dir_k[n_rows+n_cols+1: end])
     

        alpha_k_pri = min(0.99*alpha_max_pri, 1)
        alpha_k_dual = min(0.99*alpha_max_dual, 1)
        x_k += alpha_k_pri * dir_k[1:n_cols]
        lambda_k += alpha_k_dual * dir_k[n_cols+1: n_rows+n_cols]
        s_k += alpha_k_dual * dir_k[n_rows+n_cols+1: end]
        
        Xk = Diagonal(x_k)
        Sk = Diagonal(s_k)
        mu_k = s_k' * x_k / n_cols
        rb_k = A * x_k - b
        rc_k = -Q * x_k + A' * lambda_k + s_k - c
        
        # valeurs pour les conditions d'arret:
        quad_part = x_k' * Q * x_k 
        pdd = abs(quad_part + c' * x_k - b' * lambda_k ) / (1 + abs(c' * x_k + quad_part/2)) # test correct? ecart primal dual
        cond_rb = norm(rb_k) / (1 + n_b)
        cond_rc = norm(rc_k) / (1 + n_c)
        
        k += 1
        
        # affichage
        if display == true
            println("---------------------------------------------------------------")
            println("Iter ", k, ":   pdd = ", pdd, " |  cond_rb = ", cond_rb, " |  cond_rc = ", cond_rc)
        end
    end
    
    if display == true
        criteria = [k > max_iter,  pdd <= eps, cond_rb <= eps, cond_rc <= eps]
        criteria_names = ["reached max_iter",  "pdd <= eps", "cond_rb <= eps", "cond_rc <= eps"]
        println("\n stopping criterion = ",criteria_names[findall(criteria)])
    end
    
    return OrderedDict("x_opt" => x_k, "lambda_opt" => lambda_k, "s_opt" => s_k, 
        "n_iter" => k, "pdd" => pdd, "cond_rb" => cond_rb, "cond_rc" => cond_rc)
end

MehrotraPCQuad (generic function with 5 methods)

In [48]:
# probleme1
Q = [6 2 1
    2 5 2
    1 2 4]
c = [-8; -3; -3]
A = [1 0 1
    0 1 1]
b = [0; 3];

In [46]:
x01 = [1; 2; 3]
lambda01 = [3; 3]
s01 = [2; 3 ; 2]
init_vect = (x01, lambda01, s01)
res_mpc1 =  MehrotraPCQuad(Q, c, A, b, init_vect, 20);
display_results(res_mpc1)

---------------------------------------------------------------
Iter 1:   pdd = 0.36382674223402806 |  cond_rb = 0.2989622885917216 |  cond_rc = 0.37474326688574977
---------------------------------------------------------------
Iter 2:   pdd = 0.006623927996956621 |  cond_rb = 0.011868802857091215 |  cond_rc = 0.012907423249098564
---------------------------------------------------------------
Iter 3:   pdd = 0.0004020002874229886 |  cond_rb = 0.00023618917685607426 |  cond_rc = 0.0003227554841014553
---------------------------------------------------------------
Iter 4:   pdd = 1.0811623074918842e-5 |  cond_rb = 4.700164619450479e-6 |  cond_rc = 8.460364869705301e-6
---------------------------------------------------------------
Iter 5:   pdd = 2.433036907777005e-7 |  cond_rb = 9.35332759367046e-8 |  cond_rc = 1.915470394779654e-7
---------------------------------------------------------------
Iter 6:   pdd = 5.123199330905227e-9 |  cond_rb = 1.8613122036225812e-9 |  cond_rc = 4.0522

OrderedDict{String,Any} with 7 entries:
  "x_opt"      => [9.80261e-11, 3.0, 3.44927e-11]
  "lambda_opt" => [-10.7209, 12.0]
  "s_opt"      => [8.72092, 2.56284e-12, 1.72092]
  "n_iter"     => 7
  "pdd"        => 1.04764e-10
  "cond_rb"    => 3.70401e-11
  "cond_rc"    => 8.30795e-11

In [51]:
@benchmark MehrotraPCQuad(Q, c, A, b, init_vect, 20, 1e-10, 1e-2, false)

BenchmarkTools.Trial: 
  memory estimate:  971.56 KiB
  allocs estimate:  11223
  --------------
  minimum time:     397.000 μs (0.00% GC)
  median time:      405.200 μs (0.00% GC)
  mean time:        489.726 μs (15.23% GC)
  maximum time:     4.665 ms (90.09% GC)
  --------------
  samples:          10000
  evals/sample:     1